# Chrome AI Built-in Prompt API Demo

This notebook demonstrates using Chrome's built-in AI (Gemini Nano) with the Prompt API.

**Requirements:**
- Chrome browser version 128+ 
- Enable experimental flags:
  - `chrome://flags/#prompt-api-for-gemini-nano`
  - `chrome://flags/#optimization-guide-on-device-model`
- First use requires downloading the Gemini Nano model

In [ ]:
import micropip
await micropip.install("anywidget")

In [ ]:
import anywidget
import traitlets

class ChromeAIWidget(anywidget.AnyWidget):
    _esm = """
    async function render({ model, el }) {
      // Render HTML template
      function updateUI() {
        const statusColor = 
          model.get("status").includes("Error") || model.get("status").includes("not available") ? "#dc2626" :
          model.get("status").includes("Generating") || model.get("status").includes("Creating") ? "#2563eb" :
          model.get("status").includes("Ready") || model.get("status").includes("received") ? "#16a34a" :
          "inherit";
        
        el.innerHTML = `
          <div class="chrome-ai-container">
            <div class="status-message" style="color: ${statusColor}">${model.get("status")}</div>
            <textarea class="prompt-input" placeholder="Enter your prompt here..." rows="3">${model.get("input_text")}</textarea>
            <button class="submit-button">Submit</button>
            <div class="response-box">${model.get("response")}</div>
          </div>
        `;
        
        // Wire up event listeners after rendering
        const input = el.querySelector(".prompt-input");
        const button = el.querySelector(".submit-button");
        
        input.addEventListener("input", () => {
          model.set("input_text", input.value);
          model.save_changes();
        });
        
        button.addEventListener("click", submitPrompt);
        
        input.addEventListener("keydown", (e) => {
          if (e.key === "Enter" && !e.shiftKey) {
            e.preventDefault();
            submitPrompt();
          }
        });
      }
      
      // Initialize Chrome AI session
      let session = null;
      
      // Initialize UI
      updateUI();
      
      // Listen for traitlet changes and re-render
      model.on("change:status", updateUI);
      model.on("change:response", updateUI);
      model.on("change:input_text", () => {
        const input = el.querySelector(".prompt-input");
        if (input && input.value !== model.get("input_text")) {
          input.value = model.get("input_text");
        }
      });
      
      // Initialize AI session
      (async () => {
        try {
          model.set("status", "Initializing...");
          model.save_changes();
          
          if (!window.ai || !window.ai.languageModel) {
            model.set("status", "Chrome AI not available. Please enable chrome://flags/#prompt-api-for-gemini-nano");
            model.save_changes();
            el.querySelector(".submit-button").disabled = true;
            return;
          }
          
          model.set("status", "Creating AI session...");
          model.save_changes();
          session = await window.ai.languageModel.create();
          model.set("status", "Ready! Enter a prompt and press Submit or Enter.");
          model.save_changes();
          
        } catch (error) {
          model.set("status", `Error: ${error.message}`);
          model.save_changes();
          el.querySelector(".submit-button").disabled = true;
        }
      })();
      
      // Handle prompt submission
      async function submitPrompt() {
        const input = el.querySelector(".prompt-input");
        const button = el.querySelector(".submit-button");
        const promptText = input.value.trim();
        if (!promptText || !session) return;
        
        try {
          button.disabled = true;
          input.disabled = true;
          
          model.set("status", "Generating response...");
          model.set("prompt", promptText);
          model.set("response", "");
          model.save_changes();
          
          const result = await session.prompt(promptText);
          
          model.set("response", result);
          model.set("status", "Response received! Enter another prompt.");
          model.save_changes();
          
        } catch (error) {
          model.set("response", `Error: ${error.message}`);
          model.set("status", "Error occurred. Please try again.");
          model.save_changes();
        } finally {
          button.disabled = false;
          input.disabled = false;
        }
      }
    }
    export default { render };
    """
    
    _css = """
    .chrome-ai-container {
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
      max-width: 800px;
      margin: 0 auto;
      padding: 20px;
    }
    
    .status-message {
      padding: 12px;
      margin-bottom: 16px;
      border-radius: 6px;
      background-color: #f3f4f6;
      font-size: 14px;
      font-weight: 500;
    }
    
    .prompt-input {
      width: 100%;
      padding: 12px;
      margin-bottom: 12px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      font-size: 14px;
      font-family: inherit;
      resize: vertical;
      box-sizing: border-box;
    }
    
    .prompt-input:focus {
      outline: none;
      border-color: #3b82f6;
    }
    
    .submit-button {
      padding: 10px 20px;
      margin-bottom: 16px;
      background-color: #3b82f6;
      color: white;
      border: none;
      border-radius: 6px;
      font-size: 14px;
      font-weight: 500;
      cursor: pointer;
      transition: background-color 0.2s;
    }
    
    .submit-button:hover:not(:disabled) {
      background-color: #2563eb;
    }
    
    .submit-button:disabled {
      background-color: #9ca3af;
      cursor: not-allowed;
    }
    
    .response-box {
      padding: 16px;
      min-height: 100px;
      border: 2px solid #d1d5db;
      border-radius: 6px;
      background-color: #ffffff;
      white-space: pre-wrap;
      word-wrap: break-word;
      font-size: 14px;
      line-height: 1.6;
    }
    """
    
    input_text = traitlets.Unicode("").tag(sync=True)
    prompt = traitlets.Unicode("").tag(sync=True)
    response = traitlets.Unicode("Response will appear here...").tag(sync=True)
    status = traitlets.Unicode("Initializing...").tag(sync=True)

## Create and Display the Widget

Create an instance of the Chrome AI widget and display it:

In [ ]:
ai_widget = ChromeAIWidget()
ai_widget

## Access Widget State from Python

The widget uses traitlets to synchronize state between JavaScript and Python. You can access all widget properties:

In [ ]:
print("Input text:", ai_widget.input_text)
print("Last prompt:", ai_widget.prompt)
print("Last response:", ai_widget.response)
print("Status:", ai_widget.status)